In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm

session = requests.Session()

file = open('urls.csv')
data = list(csv.reader(file, delimiter=","))
file.close()

url = [item for sublist in data for item in sublist]

price, enrollment, rating, num_ratings = [], [], [], []

for URL in tqdm(url):

    try:
        req = session.get(URL).text
        soup = BeautifulSoup(req, 'html.parser')
    except:
        continue
    try:
        meta_price = soup.find("meta", {"property": "udemy_com:price"})
        price.append(meta_price.get("content", None))
    except:
        continue
    try:
        enrollment.append(soup.find("div", {"class": "enrollment"}).text)
    except:
        continue
    try:
        rating_temp = soup.find("span", {"class": "star-rating--star-wrapper--5Fj0L star-rating--medium--3Lhzz star-rating--dark-background--4E2W3"})
        rating.append(rating_temp.find("span", {"class": "ud-heading-sm star-rating--rating-number--3l80q"}).text)
    except:
        continue
    try:
        num_ratings_temp = soup.find("div", {"class": "clp-lead__element-item clp-lead__element-item--row"})
        num_ratings_temp = num_ratings_temp.find("a")
        num_ratings_temp = num_ratings_temp.find_all("span")
        num_ratings.append(num_ratings_temp[-1].text)
    except:
        continue


['https://www.udemy.com/course/microsoft-az-900-microsoft-azure-microsoft-certification/', 'https://www.udemy.com/course/aws-certified-solutions-architect-associate-saa-c03/', 'https://www.udemy.com/course/az-303-microsoft-azure-architect-technologies-exam/', '', 'https://www.udemy.com/course/aspnet-core-web-api-csharp-dotnet-course-bootcamp/', '', 'https://www.udemy.com/course/azure-devops-and-internet-of-things-essentials/', '', 'https://www.udemy.com/course/ai-900-microsoft-azure-ai-fundamentals-practice-tests-new/', '', 'https://www.udemy.com/course/preparation-a-la-certification-azure-fundamentals-az-900-v/', '', 'https://www.udemy.com/course/working-with-git-and-repos-in-azure-devops/', '', 'https://www.udemy.com/course/ai-100design-and-implementing-azure-ai-solution-prac-test/', '', 'https://www.udemy.com/course/azure-data-factory-for-azure-data-engineers-with-hands-on-labs/', '', 'https://www.udemy.com/course/6-practice-exams-azure-fundamentals-az-900/', '', 'https://www.udemy.

100%|██████████| 824/824 [04:56<00:00,  2.78it/s]


In [4]:
def cleanup(price, enrollment, rating, num_ratings):    
    info = [price, enrollment, rating, num_ratings]

    if all(len(info[0]) == len(i) for i in info[1:]):
        for i in range(0, len(price)):
            
            price[i] = price[i].strip("$")
            
            enrollment[i] = enrollment[i].strip(" studen")

            num_ratings[i] = num_ratings[i].strip("() ratings")

    return price, enrollment, rating, num_ratings

In [5]:
price, enrollment, rating, num_ratings = cleanup(price, enrollment, rating, num_ratings)

print(price, enrollment, rating, num_ratings)

['19.99', '84.99', '19.99', '84.99', '24.99', '19.99', '19.99', '84.99', '19.99', '19.99', '29.99', '19.99', '84.99', '84.99', '54.99', '19.99', '19.99', '19.99', '39.99', '54.99', '19.99', '49.99', '59.99', '19.99', '19.99', '19.99', '19.99', '49.99', '84.99', '19.99', '19.99', '84.99', '19.99', '19.99', '19.99', '19.99', '19.99', '19.99', '64.99', '19.99', '19.99', '84.99', '19.99', '84.99', '19.99', '19.99', '24.99', '19.99', '19.99', '39.99', '19.99', '54.99', '24.99', '19.99', '84.99', '39.99', '19.99', '19.99', '19.99', '49.99', '19.99', '19.99', '19.99', '84.99', '19.99', '24.99', '109.99', '39.99', '19.99', '49.99', '84.99', '24.99', '64.99', '84.99', '19.99', '19.99', '19.99', '39.99', '49.99', '19.99', '19.99', '19.99', '19.99', '84.99', '19.99', '84.99', '39.99', '19.99', '19.99', '19.99', '19.99', '19.99', '49.99', '24.99', '19.99', '19.99', '84.99', '19.99', '19.99', '84.99', '19.99', '19.99', '19.99', '19.99', '84.99', '19.99', '24.99', '19.99', '19.99', '19.99', '19.99',

In [ ]:
rows = zip(price, enrollment, rating, num_ratings)
header_list = ('price', 'enrollment', 'rating', 'num_ratings')

with open('AWS_udemy_data.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)